In [1]:
""""
ONE HOUR DATA: DIFFICULT TO PREDICT, HUMANS ARE UNPREDICTIBLE IN SHORT TERM
ONE WEEK: 
ONE MONTH DATA: SHOULD BE EASY TO PREDICT, BUT MAYBE WE DONT HAVE ENOUGH MONTHS?
""""


SyntaxError: EOL while scanning string literal (<ipython-input-1-2924865c321e>, line 5)

In [6]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm import tqdm
%matplotlib inline
plt.style.use('default')

df=pd.read_csv('/datc/opschaler/combined_gas_smart_weather_dfs/processed/P01S01W8655_hour.csv',header=0,delimiter='\t', parse_dates=['datetime'])
df=df.set_index(['datetime'])
dwelling_id = 'P01S02W4953'


In [7]:
sample_rate = '1M'
def resample_df(df, sample_rate, combine_all_dwellings=False):
    """
    Resampled a (un)processed dataframe to the specified sample_rate.
    Input is a (un)processed df.
    Input df may also be multiple dwelling dfs combined.
    Sample rate must be a string. 
    For example '1H', '1D', '60s'.
    
    Combine all dwellings: resamples the df and ignores the fact that there are unique dwellings.
    
    TODO: add std to ePower, gasPower when combine_all_dwellings=False
    """
    def resample_dwelling(df, sample_rate, dwelling_id):
        df = df.resample(sample_rate).mean() # resample to rest by mean
        df['dwelling'] = dwelling_id
        return df
        
                      
    resampled_dwellings = []
    
    if combine_all_dwellings: # Ignore dwelling_ids
        df = df.drop(['eMeter', 'eMeterReturn', 'eMeterLow', 'eMeterLowReturn', 'gasMeter'], axis=1) # Drop columns because they are meaningless when ignoring dwelling ids
        resampled_df = resample_dwelling(df, sample_rate, 'All dwellings')
        resampled_dwellings.append(resampled_df)
    else:
        dwellings = df['dwelling'].unique() # Get dwelling ids
        for dwelling_id in tqdm(dwellings):
            dwelling_df = df[df['dwelling'] == dwelling_id] # Get the data from only that dwelling_id
            resampled_dwelling = resample_dwelling(dwelling_df, sample_rate, dwelling_id)
            resampled_dwellings.append(resampled_dwelling)
    
    resampled_df = pd.concat(resampled_dwellings)
    
    return resampled_df

rdf=resample_df(df,sample_rate)
trdf = rdf[rdf['T'] < 16]

100%|██████████| 1/1 [00:00<00:00, 102.80it/s]


In [8]:
rdf=rdf[np.isfinite(rdf['gasPower'])]
trdf=trdf[np.isfinite(trdf['gasPower'])]

# # df1h =  resample_df(df, 'H')
# # df3h = resample_df(df, '3H')
# # df6h = resample_df(df, '6H')
# # df12h = resample_df(df, '12H')
# # df1d = resample_df(df, 'D')
# # df1w = resample_df(df, '7D')
# # df1m = resample_df(df, '1M')



In [9]:
# df1w=df1w[np.isfinite(df1w['gasPower'])]
# df1d=df1d[np.isfinite(df1d['gasPower'])]
# df1h=df1h[np.isfinite(df1h['gasPower'])]

# df1d
trdf

,eMeter,eMeterReturn,eMeterLow,eMeterLowReturn,ePower,ePowerReturn,gasMeter,DD,DR,FF,...,RG,SQ,T,T10,TD,U,VV,WW,gasPower,dwelling
datetime,,,,,,,,,,,,,,,,,,,,,
2017-04-30,796.895891,0.0,748.166615,0.0,391.150721,0.0,1609.625422,222.166582,35.947436,3.865200,...,0.080459,3.210664,8.045000,7.587350,2.990556,72.103205,35188.117609,12.744231,0.234331,P01S01W8655
2017-05-31,868.717199,0.0,834.528583,0.0,322.628616,0.0,1651.865533,172.348010,24.183871,3.734275,...,0.042897,3.166030,14.877061,14.647858,9.866055,73.680488,28690.985975,7.965927,0.045299,P01S01W8655
2017-09-30,1297.046715,0.0,1239.969261,0.0,321.990871,0.0,1695.154824,194.764146,66.616111,3.617731,...,0.259900,2.075907,14.199016,13.440699,11.340502,83.450417,22984.789489,17.322986,0.034571,P01S01W8655
2017-10-31,1410.374981,0.0,1345.924233,0.0,273.242882,0.0,1731.064168,229.663327,43.497581,5.212195,...,0.113515,1.400955,13.743268,13.009335,10.733477,82.127688,23593.636996,14.294086,0.073961,P01S01W8655
2017-11-30,1519.645467,0.0,1428.951309,0.0,374.991721,0.0,1816.156442,224.729098,46.639158,3.652667,...,0.091158,1.030099,8.126218,7.311656,5.863011,85.817439,20512.914818,17.210105,0.240375,P01S01W8655


In [18]:
#def create_corr_matrix(df, dwelling_id, annot):
    """
    Pearson correlation coefficient matrix. 
    The Pearson correlation coefficient is a measure of the linear correlation between two variables.
    """
 #   plt.clf()
 #   %matplotlib inline
  #  corr = df.corr()
  #  mask = np.zeros_like(df.corr())
  #  mask[np.triu_indices_from(mask)] = True

  #  if annot:
    #    fig, ax = plt.subplots(figsize=(25,25))
  #  else:
   #     fig, ax = plt.subplots()

  #  fig = sns.heatmap(corr, mask=mask, square=False, cmap='RdYlGn', annot=annot, ax=ax, 
     #           cbar_kws={'label':'Pearson correlation coefficient [-]'})
#
#    fig.set_title('Correlation matrix of dwelling ID: '+dwelling_id)
#    fig.tick_params(axis='x', rotation=90)
 #   fig.tick_params(axis='y', rotation=0)

 #   fig = fig.get_figure()
#    fig.tight_layout()
    
#    fig.show()
    
#    return fig

IndentationError: unexpected indent (<ipython-input-18-450727274783>, line 2)

In [11]:
#fig = create_corr_matrix(df1m, dwelling_id, True)

NameError: name 'df1m' is not defined

In [12]:
from sklearn.model_selection import train_test_split

X = trdf[['T']]
Y = trdf['gasPower']

X = np.array(X).reshape(-1,len(X.columns))
y = np.array(Y).reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

model = sm.OLS(X_train, y_train).fit()
y_prediction=model.predict(X_test)
print_model = model.summary()

print(print_model)
print('\n==============================================================================\nCOEFFICIENTES\n------------------------------------------------------------------------------\n ')
print(model.params)
print('\n==============================================================================\nMEAN SQUARED ERROR\n------------------------------------------------------------------------------\n ')
print(model.mse_total)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.411
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     1.393
Date:                Mon, 15 Oct 2018   Prob (F-statistic):              0.359
Time:                        10:32:00   Log-Likelihood:                -10.997
No. Observations:                   3   AIC:                             23.99
Df Residuals:                       2   BIC:                             23.09
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
gasPower      55.0946     46.675      1.180      0.3

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/statsmodels/stats/stattools.py:72: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  "samples were given." % int(n), ValueWarning)


In [13]:
[b,a]=model.params
print('Y = '+str(round(a,3))+'*T + '+str(round(b,3)))

ValueError: not enough values to unpack (expected 2, got 1)

In [14]:
xx=trdf['T']
yy=a*xx+b

plt.plot(rdf['T'],rdf['gasPower'],'ob',xx,yy,'-r')
plt.grid(True)
plt.xlabel('Temperature [°C]')
plt.ylabel('gasPower [m$^3$ per hour]')
plt.show()

NameError: name 'a' is not defined

In [15]:
xx=df1w['T'][10:20]
yy=a*xx+b

plt.plot(df1w['T'][0:10],df1w['gasPower'][0:10],'b^',label='TRAINED DATA')
plt.plot(xx,yy,'rx',label='PREDICTED DATA')     
plt.plot(df1w['T'][10:20],df1w['gasPower'][10:20],'go',label='TEST DATA')

plt.grid(True)
plt.xlabel( 'Temperature [°C]')
plt.ylabel('gasPower [m$^3$ per hour]')

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

NameError: name 'df1w' is not defined